In [1]:
import numpy as np
import pandas as pd

**Преобразуем тестовую выборку из соревнования в стандартный вид, который мы использовали в обучающей выборке.**

In [2]:
test_df = pd.read_csv('kaggle_data/test_sessions.csv', index_col='session_id', \
                      parse_dates=[1,3,5,7,9,11,13,15,17,19], \
                      infer_datetime_format=True)

In [3]:
test_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,1917,2014-12-04 20:36:21,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN
2,20225,2014-03-04 14:08:55,27,2014-03-04 14:08:59,9,2014-03-04 14:08:59,32,2014-03-04 14:09:50,20225,2014-03-04 14:10:43,20258,2014-03-04 14:10:44,20225,2014-03-04 14:10:45,20225,2014-03-04 14:11:07,20276,2014-03-04 14:11:08,20225,2014-03-04 14:11:09
3,71,2014-03-31 09:20:38,63,2014-03-31 09:20:42,64,2014-03-31 09:20:43,63,2014-03-31 09:20:43,71,2014-03-31 09:20:43,22,2014-03-31 09:20:45,64,2014-03-31 09:20:46,71,2014-03-31 09:20:47,71,2014-03-31 09:20:48,70,2014-03-31 09:20:51
4,7338,2014-02-14 13:39:36,7338,2014-02-14 14:07:53,7338,2014-02-14 14:08:36,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN
5,19622,2014-03-31 09:46:04,32,2014-03-31 09:46:12,340,2014-03-31 09:46:13,19634,2014-03-31 09:46:14,1721,2014-03-31 09:46:14,327,2014-03-31 09:46:15,38,2014-03-31 09:46:19,1721,2014-03-31 09:46:19,340,2014-03-31 09:46:20,19634,2014-03-31 09:46:20


In [4]:
def get_site_index(csv_file):
    site_index = pd.read_csv(csv_file, header=None, names=['siteid', 'site'], index_col=0)
    site_dic = site_index.to_dict("dic")
    return site_dic["site"]

In [5]:
data = []
site_dic = get_site_index("kaggle_data/site_indexes.txt")
for session in test_df.values:
    for i in range (0, 20, 2):
        if str(session[i]) != 'nan':
            row = [session[i+1], site_dic[int(session[i])]]
            data.append(row)

In [6]:
data = pd.DataFrame(data, columns=["timestamp", "site"])

In [8]:
data.to_csv("kaggle_data/full_test_temp.csv", index=False)

**А теперь займёмся самим соревнованием.**

In [1]:
def write_to_submission_file(predicted_labels, out_file,
                             target='user_id', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [9]:
from scipy.sparse import csr_matrix, coo_matrix, hstack
def sparsematrix(X, skip_zero=True, skip_popular=False, top_sites = {}):
    indptr = [0]
    indices = []
    data = []
    vocabulary = {}
    for d in X:
        for term in d:
            if (skip_zero and term != 0) or not skip_zero:
                index = vocabulary.setdefault(term, len(vocabulary))
                indices.append(index)
                data.append(1)
        indptr.append(len(indices))
    return csr_matrix((data, indices, indptr), dtype=int)

In [377]:
import pickle
pkl_file = open('kaggle_data/full_train.pkl', 'rb')
train_data = pickle.load(pkl_file)

pkl_file = open('kaggle_data/full_test.pkl', 'rb')
test_data = pickle.load(pkl_file)

In [93]:
train_data.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,...,#unique_sites,site_longest_time,start_hour,day_of_week,daytime,fb_portion,youtube_portion,top30_portion,bot30_portion,target
0,3537,3537,3537,5759,37,3537,16,17,47,53,...,7,5759,14,4,1,0,0,0,0,280
1,37,56,47,3537,53,47,53,17,22,6,...,8,22,14,4,1,0,0,0.939394,0,280
2,22,22,22,90,22,22,87,22,14326,14326,...,4,87,14,4,1,0,0,0.5,0,280
3,14326,87,14334,14326,0,0,0,0,0,0,...,3,87,14,4,1,0,0,0,0,280
4,540,16,56,47,47,53,17,1027,347,16,...,8,17,15,4,1,0,0,0,0,280


###########################################

***Попробуем сделать новые признаки.***

In [378]:
pkl_file = open("kaggle_data/site_freq.pkl", 'rb')
site_freq = pickle.load(pkl_file)
top_sites = [v[1] for k, v in sorted(site_freq.items(), key=lambda t: t[1][1], reverse=True)[:0]]
print top_sites

[]


In [282]:
train_data["prediction"] = 0

In [283]:
train_data.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,...,site_longest_time,start_hour,day_of_week,daytime,fb_portion,youtube_portion,top30_portion,bot30_portion,target,prediction
0,3537,3537,3537,5759,37,3537,16,17,47,53,...,5759,14,4,1,0,0,0,0,280,0
1,37,56,47,3537,53,47,53,17,22,6,...,22,14,4,1,0,0,0.939394,0,280,0
2,22,22,22,90,22,22,87,22,14326,14326,...,87,14,4,1,0,0,0.5,0,280,0
3,14326,87,14334,14326,0,0,0,0,0,0,...,87,14,4,1,0,0,0,0,280,0
4,540,16,56,47,47,53,17,1027,347,16,...,17,15,4,1,0,0,0,0,280,0


In [379]:
user_dic = {} #dictionary with keys as user_ids and values are sites with number of visits by the given user
site_dic = {} #dictionary with keys as sites and values are unique user_ids who visited that website at least once.

for i, v in train_data.iterrows():
    if v.target not in user_dic:
        user_dic[v.target] = {}
    for site in ['site' + str(i) for i in range(1,11)]:
        if v[site] != 0 and v[site] not in top_sites:
            if v[site] in user_dic[v.target]:
                user_dic[v.target][v[site]] +=1
            else:
                user_dic[v.target][v[site]] = 1
        
        if v[site] in site_dic:
            site_dic[v[site]].add(v.target)
        else:
            site_dic[v[site]] = set([v.target])

**Let's check if there are websites that are visited by only one person. If yes, then if a session contains such website, we'll consider the whole session belonging to that user.**

In [385]:
%%time

for data in [test_data, train_data]:
    data["prediction"] = 0
    for r, v in data.iterrows():
        session_prediction = []
        for site in ['site' + str(c) for c in range(1,11)]:
            if v[site] in site_dic and len(site_dic[v[site]]) == 1:
                data.set_value(r, -1, list(site_dic[v[site]])[0], takeable=True)
                continue
            elif v[site] in site_dic and len(site_dic[v[site]]) < 1:
                predictions = sorted([[int(user), float(user_dic[user][v[site]]) / sum(user_dic[user].values())] \
                                     for user in site_dic[v[site]] if v[site] in user_dic[user]], key=lambda t: t[1], reverse=True)
                if len(predictions):
                    session_prediction.append(predictions[0])
        if v["prediction"] == 0 and len(session_prediction):
            prediction = sorted(session_prediction, key=lambda t: t[1], reverse=True)[0][0]
            data.set_value(r, -1, prediction, takeable=True)

CPU times: user 43.6 s, sys: 204 ms, total: 43.8 s
Wall time: 43.5 s


In [386]:
total_filled = len(train_data[train_data.prediction != 0]) / float(len(train_data))
total_filled

0.20320990635943487

In [387]:
right_filled = sum([int(int(a) == int(b)) for a, b in zip(train_data["target"], train_data["prediction"])]) / float(len(train_data))
right_filled

0.20320990635943487

In [388]:
right_filled/total_filled

1.0

In [389]:
accuracy_score(train_data["target"].astype(int), train_data["prediction"].astype(int))

0.20320990635943487

In [393]:
#Number of websites with only one particular visiting user.
a = len([k for k, v in sorted([(k, len(v)) for k, v in site_dic.items()], key=lambda t: t[1]) if v ==1])
print a, float(a)/len(site_dic) 

13732 0.637718850137


In [135]:
top_user_site = {}
for user in user_dic:
    top_user_site[user] = [k for k, v in sorted(user_dic[user].items(), key=lambda t: t[1], reverse=True)[:50]]
    #print user
    #print [(k, round(float(v)/sum(user_dic[user].values()) *100, 1)) for k, v in sorted(user_dic[user].items(), key=lambda t: t[1], reverse=True)[:1]]
    #print [k for k, v in sorted(user_dic[user].items(), key=lambda t: t[1], reverse=True)[:10]]

############################################################

In [336]:
train_test_data_sites = pd.concat([train_data[['site' + str(i) for i in range(1,11)]], \
                                   test_data[['site' + str(i) for i in range(1,11)]]])

In [6]:
train_test_data_sites = pd.concat([train_data[['site' + str(i) for i in range(1,11)] + ['site_longest_time']], \
                                   test_data[['site' + str(i) for i in range(1,11)] + ['site_longest_time']]])

In [116]:
#another attempt with features
train_test_data_sites = pd.concat([train_data[['site' + str(i) for i in range(1,11)] + ["#unique_sites", "day_of_week", "daytime"]], \
                                   test_data[['site' + str(i) for i in range(1,11)] + ["#unique_sites", "day_of_week", "daytime"]]])

In [63]:
#another attempt with features
train_test_data_sites = pd.concat([train_data[['site' + str(i) for i in range(1,11)] + ["#unique_sites", "day_of_week", "start_hour"]], \
                                   test_data[['site' + str(i) for i in range(1,11)] + ["#unique_sites", "day_of_week", "start_hour"]]])

In [10]:
sparsematrix(train_test_data_sites.as_matrix())

<132270x24052 sparse matrix of type '<type 'numpy.int64'>'
	with 1351142 stored elements in Compressed Sparse Row format>

In [78]:
#X_train_test_sparse = sparsematrix(train_test_data_sites.as_matrix(), skip_popular=True, top_sites = top_sites)
X_train_test_sparse = sparsematrix(train_test_data_sites.as_matrix())
X_train_sparse = X_train_test_sparse[:len(train_data)]
X_test_sparse = X_train_test_sparse[len(train_data):]
y = train_data["target"]

In [201]:
train_test_data_sites.shape

(132270, 10)

In [202]:
X_train_test_sparse.shape

(132270, 23584)

In [11]:
train_features = coo_matrix(train_data[["session_timespan", "#unique_sites", "day_of_week", "daytime", "fb_portion", "youtube_portion",\
           "top30_portion", "bot30_portion"]].values, dtype=float)
test_features = coo_matrix(test_data[["session_timespan", "#unique_sites", "day_of_week", "daytime", "fb_portion", "youtube_portion",\
           "top30_portion", "bot30_portion"]].values, dtype=float)

In [77]:
train_features = coo_matrix(train_data[["#unique_sites", "fb_portion", "youtube_portion", "top30_portion"]].values, dtype=float)
test_features = coo_matrix(test_data[["#unique_sites", "fb_portion", "youtube_portion", "top30_portion"]].values, dtype=float)
#These features scored a little bit lower than pure websites.

In [198]:
train_features = coo_matrix(train_data[["#unique_sites", "day_of_week", "daytime"]].as_matrix(), dtype=int)
test_features = coo_matrix(test_data[["#unique_sites", "day_of_week", "daytime"]].as_matrix(), dtype=int)
#These features сurrently hold a record of 0.39115 on Kaggle test set.

In [337]:
train_features = coo_matrix(train_data[["#unique_sites", "day_of_week", 'start_hour', "prediction"]].as_matrix(), dtype=int)
test_features = coo_matrix(test_data[["#unique_sites", "day_of_week", 'start_hour', "prediction"]].as_matrix(), dtype=int)

In [179]:
train_features.toarray().shape

(91093, 3)

In [203]:
X_train_sparse.shape

(91093, 23584)

In [181]:
X_train_sparse

<91093x23957 sparse matrix of type '<type 'numpy.int64'>'
	with 852946 stored elements in Compressed Sparse Row format>

In [338]:
X_train_sparse = hstack([X_train_sparse, train_features])
X_test_sparse = hstack([X_test_sparse, test_features])

In [81]:
X_train_sparse.shape

(91093, 24055)

In [339]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_sparse, y, test_size=0.3, 
                                                     random_state=7, stratify=y)

In [340]:
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, f1_score

In [341]:
skf = StratifiedKFold(n_splits=3, shuffle=True)

In [161]:
sgd_svm = SGDClassifier(n_jobs=-1)

In [162]:
%%time
svm_params1 = {'n_iter': [5], 'loss': ['log', 'hinge', 'squared_hinge'], 'penalty': ['l2','l1','elasticnet']}

svm_grid_searcher1 = GridSearchCV(sgd_svm, param_grid=svm_params1, cv=skf, n_jobs=-1)
svm_grid_searcher1.fit(X_train, y_train)

CPU times: user 47.1 s, sys: 208 ms, total: 47.3 s
Wall time: 4min 6s


In [163]:
print(svm_grid_searcher1.best_params_)
print(svm_grid_searcher1.best_score_)

{'penalty': 'l1', 'loss': 'squared_hinge', 'n_iter': 5}
0.209503646201


In [164]:
pred = svm_grid_searcher1.predict(X_valid)

In [165]:
print(accuracy_score(y_valid, pred))

0.234009074941


**Попробуем снова LinearSVC с оптимальным параметром С, полученным на прошлой неделе.**

In [342]:
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel

In [343]:
%%time
#"day_of_week", 'start_hour'
#svm = LinearSVC(C=0.069896551724137931, dual=False, penalty='l1')
svm = LinearSVC(C=0.10434482758620689, dual=True, penalty='l2')
#svm_params2 = {'C': [0.069896551724137931, 0.094210526315789467, 0.10434482758620689]}
#svm_params2 = {'C': np.linspace(1e-3, 1, 20, 'dual':[True], 'penalty': ['l1']}

svm.fit(X_train, y_train)

#svm_grid_searcher2 = GridSearchCV(svm, param_grid=svm_params2, cv=skf, n_jobs=-1)
#svm_grid_searcher2.fit(X_train, y_train)

CPU times: user 1h 58s, sys: 284 ms, total: 1h 58s
Wall time: 1h 58s


In [344]:
accuracy_score(y_valid, svm.predict(X_valid))

0.20005122950819673

In [86]:
model = SelectFromModel(svm, prefit=True)
X_train_new = model.transform(X_train)
print X_train_new.shape
X_valid_new = model.transform(X_valid)
print X_valid_new.shape

(63765, 4088)
(27328, 4088)


In [89]:
accuracy_score(y_valid, svm.predict(X_valid_new))

0.3271735948477752

In [87]:
accuracy_score(y_valid, svm.predict(X_valid))

0.2968384074941452

In [40]:
print(svm_grid_searcher2.best_params_)
print(svm_grid_searcher2.best_score_)

{'C': 0.1061578947368421, 'dual': True}
0.321336156199


In [41]:
pred2 = svm_grid_searcher2.predict(X_valid)

In [42]:
print(accuracy_score(y_valid, pred2))

0.333613875878


In [23]:
write_to_submission_file(svm_grid_searcher2.predict(X_test_sparse), "kaggle_data/submission9.csv")

#LinearSVC score=0.38950, validation score=0.352641978923
#C=0.1061578947368421
#"site_longest_time", "#unique_sites", "day_of_week", "start_hour"

#Removing top 100 websites from the train and test sets didn't help
#LinearSVC showed validation score=0.354069086651 and kaggle score = 0.38843
#"#unique_sites", "day_of_week", "start_hour"

# Current winner with score=0.39368. Validation score=0.358121546961
# LinearSVC, C=0.10434482758620689, features = #"#unique_sites", "day_of_week", "start_hour"

# Logistic Regression with C = 0.6899655172413793, 
# one-vs-rest, features="#unique_sites", "day_of_week", 'start_hour', score=0.38318

# Current winner with score=0.39115
# LinearSVC, C=0.10434482758620689, features = #"#unique_sites", "day_of_week", "daytime"

# The first winning attempt with score=0.37090:
# LinearSVC, C=0.069896551724137931, no features, original train data.